#BERT (Basic Encoder-Decoder) model running for both BERT(pre-trained) and BERT Summary and evaluating on CNN/DailyMail test dataset.

In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install -U py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 640.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  U

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cnn_dailymail", version = '3.0.0', split='test',trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

In [ ]:
!pip install bert-score
!pip install rouge-score
import torch
from datasets import load_dataset, load_metric
from transformers import BertTokenizer, EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_score import score as bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=01392119dd9a903d0339875b5aef37426d307d30c15a636cd8ebfc358d51b01d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
#initializing for BERT and BERTSUM models and its tokenizers
models = {
    "BERT (Basic Encoder-Decoder)": EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased"),
    "BERTSUM": AutoModelForSeq2SeqLM.from_pretrained("Shobhank-iiitdwd/BERT_summary"),
}
tokenizers = {
    "BERT (Basic Encoder-Decoder)": BertTokenizer.from_pretrained("bert-base-uncased"),
    "BERTSUM": AutoTokenizer.from_pretrained("Shobhank-iiitdwd/BERT_summary"),
}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
for model in models.values():
    model.to(device)

#initializing the evaluation metrics
rouge = load_metric('rouge', trust_remote_code=True)
meteor = load_metric('meteor', trust_remote_code=True)
#based on the GPU-RAM available on runtime 3, using A100 GPU, sufficient memmory was there to support this batch size configuration
batch_size = 16
results = []

# processing of dataset batchwise
for model_name in models.keys():
    reference_summaries =[]
    generated_summaries =[]

    model = models[model_name]
    tokenizer = tokenizers[model_name]
    print(f"Processing {model_name}")
    #starttoken is set to padtoken id to overcome the decoder issue
    decoder_start_token_id = tokenizer.pad_token_id
    # batch processing
    for i in tqdm(range(0, len(dataset), batch_size), desc=f"Processing Test Set with {model_name}"):
        batch = dataset.select(range(i, min(i + batch_size, len(dataset))))

        # Access the articles and highlights straighy from the batch dictionary
        articles = batch['article']
        references =batch['highlights']
        # Tokenize and encode the articles with left padding
        inputs = tokenizer(
            articles,
            max_length=512,
            return_tensors='pt',
            truncation=True,
            padding='longest',  #left padding is used to set with the encoder decoder setup
            pad_to_multiple_of=512
        )
        inputs = inputs.to(device)
        summary_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # using th attention mask for proper decoding
            decoder_start_token_id=decoder_start_token_id,
            num_beams=4,
            max_length=142,
            early_stopping=True
        )
        generated_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

        #storing highlights and generated summaries
        reference_summaries.extend([nltk.word_tokenize(ref.lower()) for ref in references])
        generated_summaries.extend([nltk.word_tokenize(gen.lower()) for gen in generated_batch])

        #updating ROUGE and METEOR score metrics
        rouge.add_batch(predictions=generated_batch, references=references)
        meteor.add_batch(predictions=generated_batch, references=references)

    #bleuscore
    bleu_score = corpus_bleu([[ref] for ref in reference_summaries], generated_summaries, smoothing_function=SmoothingFunction().method7)
    #rougescore
    rouge_result = rouge.compute()
    #meteorscore
    meteor_result = meteor.compute()
    #cosine similarity
    vectorizer= TfidfVectorizer().fit(references + generated_batch)
    reference_vectors= vectorizer.transform(references)
    generated_vectors= vectorizer.transform(generated_batch)
    cosine_scores= cosine_similarity(reference_vectors, generated_vectors).diagonal().mean()
    # BERTScore
    P, R, F1 =bert_score(generated_batch, references, lang='en', verbose=True)
    bert_score_f1= F1.mean().item()
    #results
    results.append({
        "Model": model_name,
        "BLEU": bleu_score,
        "ROUGE-1": rouge_result['rouge1'].mid.fmeasure,
        "ROUGE-2": rouge_result['rouge2'].mid.fmeasure,
        "ROUGE-L": rouge_result['rougeL'].mid.fmeasure,
        "METEOR": meteor_result['meteor'],
        "Cosine Similarity": cosine_scores,
        "BERTScore": bert_score_f1
    })

print(df)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 29.46 sentences/sec
                          Model      BLEU   ROUGE-1       ROUGE-2   ROUGE-L  \
0  BERT (Basic Encoder-Decoder)  0.065266  0.002066  9.670245e-07  0.002064   
1                       BERTSUM  0.237870  0.397653  1.811645e-01  0.275032   

     METEOR  Cosine Similarity  BERTScore  
0  0.015882           0.000000   0.703756  
1  0.340458           0.409992   0.857486  


In [ ]:
df

,Model,BLEU,ROUGE-1,ROUGE-2,ROUGE-L,METEOR,Cosine Similarity,BERTScore
0,BERT (Basic Encoder-Decoder),0.065266,0.002066,9.670245e-07,0.002064,0.015882,0.000000,0.703756
1,BERTSUM,0.237870,0.397653,1.811645e-01,0.275032,0.340458,0.409992,0.857486
